In [25]:
CONFIG_FILE_PATH_PREFIX = 'cfg/'
CONFIG_TMP_FILE_PATH_PREFIX = 'cfg/tmp/'
CONFIG_BACKUP_FILE_PATH_PREFIX = 'cfg/backup/'

OUTPUT_FILE_PATH_PREFIX = 'output/'
OUTPUT_TMP_FILE_PATH_PREFIX = 'output/tmp/'

CONFIG_NUMBER = 10000

In [3]:
#!pip install Faker

In [4]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

22/11/22 13:19:16 WARN Utils: Your hostname, htknphuong-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.142.129 instead (on interface ens33)
22/11/22 13:19:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/22 13:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/22 13:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/22 13:19:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
##Reading the config from file for max id
import os.path
idConfig = {
    "accountid": 10000000,
    "defaultaccountid": 10000000,
    "transactionid": 0
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        if (file not in ["transactionid"]):
            print ('Please run CustomerFactGenerator first')
            break
        print(file + ' is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()

-----------------
accountid is exist
begin loadingaccountid
defaultaccountid is exist
begin loadingdefaultaccountid
transactionid is not exist
begin to creating transactionid


In [6]:
print(idConfig)

{'accountid': 10001004, 'defaultaccountid': 10000000, 'transactionid': 0}


In [16]:
import random
import faker
from datetime import datetime
import re
class TransactionObject:
    TypeTransactionDim = {
        1: "cash withdrawals",
        2: "online payments",
        3: "debit card charges",
        4: "wire transfers",
    }
    ConcurrencyDim = {
        1: "VND",
        2: "USD",
        3: "SIG"
    }
    TransactionPointDim = {
        1: "Banking App",
        2: "ATM",
        3: "Bank branch"
    }
    fake = faker.Faker()

    def __init__(self):
        self.transactionid = idConfig["accountid"]
        idConfig["accountid"] += 1
        self.amount = random.randint(1, 10000)
        self.timestamp = TransactionObject.fake.date()
        __concurrency = random.choice(list(TransactionObject.ConcurrencyDim.keys()))
        self.concurrency = TransactionObject.ConcurrencyDim[__concurrency]
        __transactionpoint = random.choice(list(TransactionObject.TransactionPointDim.keys()))
        self.transactionpoint = TransactionObject.TransactionPointDim[__transactionpoint]
        
        __typetransaction = random.choice(list(TransactionObject.TypeTransactionDim.keys()))
        self.typetransaction = random.choice(list(TransactionObject.TypeTransactionDim.keys()))
            
        self.destinationaccountid = random.randint(idConfig['defaultaccountid'], idConfig['accountid'])
        self.accountid = random.randint(idConfig['defaultaccountid'], idConfig['accountid'])
        
        self.exchangerate = 1
        self.lattitude = random.uniform(-85, 85)
        self.longtitude = random.uniform(-180, 180)


    def toList(self):
        return [Row(
            self.transactionid,
            self.amount,
            self.timestamp,
            self.typetransaction,
            self.concurrency,
            self.transactionpoint,
            self.destinationaccountid,
            self.accountid,
            self.exchangerate,
            self.lattitude,
            self.longtitude
            )]


In [26]:
TransactionAccount = []
for x in range(CONFIG_NUMBER):
    Transaction = TransactionObject().toList()
    TransactionAccount = TransactionAccount + Transaction

In [27]:
FactTransactionColumns = StructType([
    StructField("transactionid", StringType(), True),
    StructField("amount", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("typetransaction", StringType(), True),
    StructField("concurrency", StringType(), True),
    StructField("transactionpoint", StringType(), True),
    StructField("destinationaccountid", StringType(), True),
    StructField("accountid", StringType(), True),
    StructField("exchangerate", StringType(), True),
    StructField("lattitude", StringType(), True),
    StructField("longtitude", StringType(), True)
    ])

In [28]:
MainDf = spark.createDataFrame(data=TransactionAccount, schema = FactTransactionColumns)

In [29]:
from datetime import datetime
now = datetime.now()
fileName = 'FactTransaction ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [30]:
print(fileName)

FactTransaction 2022-11-22 14-47-43-251365


In [31]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

In [32]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

begin to rewrite accountid
-----------------
begin to rewrite defaultaccountid
-----------------
begin to rewrite transactionid
-----------------


In [33]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)

'output/FactTransaction 2022-11-22 14-47-43-251365'